## Creating dates

What is the problem?

We have the data melted from the excel files. However we do not have a date.

Currently we have the week number and abbreviated day of week. We can get hte year from the filename.

We need to reverse engineer the date. 

We can use strptime

We need to create a column with the week number (which we get from the sheet name)
*WARNING in hte first week of the year there is a potential to have week 53 and week 1. 53 would be from the previous year.



In [1]:
import pandas as pd

In [2]:
files = !ls output/*csv*

In [3]:
files

['output/dublinfootfall2014.csv',
 'output/pedestrianfootfall2007.csv',
 'output/pedestrianfootfall2008.csv',
 'output/pedestrianfootfall2009.csv',
 'output/pedestrianfootfall2010.csv',
 'output/pedestrianfootfall2011.csv',
 'output/pedestrianfootfall2012.csv',
 'output/pedestrianfootfall2013.csv']

In [4]:
#TODO strip the year using regex and create the output file name from that
file = files[0]

In [5]:
df = pd.read_csv(file)

In [6]:
df.head()

,Sheet Number,Entrance,Time,Day,Direction,Value
0,Week 1 (2014),O'Connell Street at Clerys,00:00:00,Mon,In,181
1,Week 1 (2014),O'Connell Street at Clerys,01:00:00,Mon,In,46
2,Week 1 (2014),O'Connell Street at Clerys,02:00:00,Mon,In,22
3,Week 1 (2014),O'Connell Street at Clerys,03:00:00,Mon,In,5
4,Week 1 (2014),O'Connell Street at Clerys,04:00:00,Mon,In,7


In [7]:
file

'output/dublinfootfall2014.csv'

In [8]:
#use some regex to get the year from the filename
import re
get_year = re.compile('\d{4}', re.IGNORECASE)
year = get_year.search(file)
print(year[0])
df['year'] = year[0]

2014


In [9]:
df.head()

,Sheet Number,Entrance,Time,Day,Direction,Value,year
0,Week 1 (2014),O'Connell Street at Clerys,00:00:00,Mon,In,181,2014
1,Week 1 (2014),O'Connell Street at Clerys,01:00:00,Mon,In,46,2014
2,Week 1 (2014),O'Connell Street at Clerys,02:00:00,Mon,In,22,2014
3,Week 1 (2014),O'Connell Street at Clerys,03:00:00,Mon,In,5,2014
4,Week 1 (2014),O'Connell Street at Clerys,04:00:00,Mon,In,7,2014


In [10]:
def get_week(vals):
    #pattern = r'\d+'
    pattern = r'((Week)|(week))(\_|\s)(\d+)'
    #x = re.findall(pattern, vals)
    x = re.match(pattern, vals)
    if x :
        return(x[0])

df['week_number'] = df['Sheet Number'].apply(get_week)
df['week_number'] = df['week_number'].str.strip('keeWw ')

In [11]:
df.tail()

,Sheet Number,Entrance,Time,Day,Direction,Value,year,week_number
148507,Week 34 (2014),South William St,19:00:00,Sun,Out,259,2014,34
148508,Week 34 (2014),South William St,20:00:00,Sun,Out,123,2014,34
148509,Week 34 (2014),South William St,21:00:00,Sun,Out,108,2014,34
148510,Week 34 (2014),South William St,22:00:00,Sun,Out,66,2014,34
148511,Week 34 (2014),South William St,23:00:00,Sun,Out,49,2014,34


## Issue with abbreviated days

The issue encountered here was days were as such::
{'Fri', 'Mon', 'Sat', 'Sun', 'Thurs', 'Tues', 'Weds'}

When they should be::
['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

Created a replace map to update the values

In [12]:
import calendar
abbreviated_days = list(calendar.day_abbr)

In [13]:
abbreviated_days

['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

In [14]:
set(df['Day'])

{'Fri', 'Mon', 'Sat', 'Sun', 'Thurs', 'Tues', 'Weds'}

In [15]:
""" we only need to update The,Tue and Wed """
replace_map = {'Mon':'Mon','Tues':'Tue','Weds':'Wed','Thurs':'Thu','Fri':'Fri','Sat':'Sat','Sun':'Sun'}

df['day']=df['Day'].map(replace_map)

In [16]:
df = df.drop('Day', axis=1)

In [17]:
df['day'].value_counts()

Thu    21216
Sat    21216
Tue    21216
Mon    21216
Fri    21216
Wed    21216
Sun    21216
Name: day, dtype: int64

In [18]:
df['raw_date'] = df['day'] +' '+df['week_number']+' '+df['year']

In [20]:
from datetime import datetime

"""
We are going to convert to a date from Raw date which is e.g. Mon 1 2007("%a %W %Y")
%a - Abbreviated Day e.g. Mon/Tue
%W - Week Number (starting 1 at first Monday)
%Y - Year

This will output a date
"""

def convert_to_date(vals):
    date_object = datetime.strptime(vals, "%a %W %Y")
    #print(date_object)
    if date_object:
        return(date_object)

In [21]:
df['date'] = df['raw_date'].apply(convert_to_date)

TypeError: strptime() argument 1 must be str, not float

In [22]:
df.tail()

,Sheet Number,Entrance,Time,Direction,Value,year,week_number,day,raw_date
148507,Week 34 (2014),South William St,19:00:00,Out,259,2014,34,Sun,Sun 34 2014
148508,Week 34 (2014),South William St,20:00:00,Out,123,2014,34,Sun,Sun 34 2014
148509,Week 34 (2014),South William St,21:00:00,Out,108,2014,34,Sun,Sun 34 2014
148510,Week 34 (2014),South William St,22:00:00,Out,66,2014,34,Sun,Sun 34 2014
148511,Week 34 (2014),South William St,23:00:00,Out,49,2014,34,Sun,Sun 34 2014


In [23]:
df = df.drop(['Sheet Number','year','raw_date','week_number'], axis=1)

In [24]:
df = df.sort_values(by=['date','Time','Direction','Entrance'])

KeyError: 'date'

In [174]:
output_path = 'final_output/'
output_file = 'pedestrian-footfall-data'+year[0]+'.csv'

In [175]:
df.to_csv(output_path+output_file, index=False)

# Conlcusion

Tidy up

#TODO
* merge this clean up file into the previous one
* complete all into the one
* update the output folder
* rename the first sheet of the tricky 2014 one
* 
* screenshots from tableau/power BI
* pust to git
* the 2014 file is problematic
* the 2012 file is problematic with the sheet name begging with the year (using stronger regex breaks further down the road.
* leave it for today and revisit

